# Reto 03-C-Embeddings 

## 1. Descripción General 

En el último desafío (`03-B-Chunking`), trabajamos para entender los límites de tokens con LLM y utilizar la segmentación. Ahora, si hay gigabytes de datos, también tendremos muchos fragmentos por crear. ¿Hay alguna manera de seleccionar los fragmentos de texto más relevantes? La respuesta es sí. Para resolver este problema, podemos observar un proceso llamado Embeddings. Embeddings nos ayuda a crear representaciones numéricas para todos los fragmentos. Luego, podemos encontrar los fragmentos más similares en la lista de embeddings. Una forma popular de encontrar los fragmentos similares es mediante la similitud del coseno.

### **Descripción general de Embeddings**

Un embedding es un formato especial de representación de datos que puede ser fácilmente utilizado por modelos y algoritmos de machine learning. El embedding es una representación densa en información del significado semántico de un fragmento de texto. Cada embedding es un vector de números de punto flotante, de modo que la distancia entre dos embeddings en el espacio vectorial está correlacionada con la similitud semántica entre dos entradas en el formato original. Por ejemplo, si dos textos son similares, entonces sus representaciones vectoriales también deberían ser similares.

Diferentes modelos de embedding de Azure OpenAI están específicamente creados para ser buenos en tareas particulares:
- Los embeddings de similitud son buenos para capturar la similitud semántica entre dos o más fragmentos de texto.
- Los embeddings de búsqueda de texto ayudan a encontrar qué documento largo es relevante para una consulta corta.
- Los embeddings de búsqueda de código son útiles para incrustar fragmentos de código y consultas de búsqueda en lenguaje natural.

Los embeddings facilitan hacer machine learning en grandes entradas que representan palabras al capturar las similitudes semánticas en un espacio vectorial. Por lo tanto, podemos usar embeddings para determinar si dos fragmentos de texto están semánticamente relacionados o son similares, y de manera inherente proporcionar una puntuación para evaluar la similitud.

### **Similitud del Coseno**
Un enfoque utilizado anteriormente para emparejar documentos similares se basaba en contar el número máximo de palabras comunes entre documentos. Esto es defectuoso ya que, a medida que aumenta el tamaño del documento, aumenta la superposición de palabras comunes incluso si los temas difieren. Por lo tanto, la similitud del coseno es un mejor enfoque.

Matemáticamente, la similitud del coseno mide el coseno del ángulo entre dos vectores proyectados en un espacio multidimensional. Esto es beneficioso porque si dos documentos están muy separados por la distancia euclidiana debido al tamaño, aún podrían tener un ángulo más pequeño entre ellos y, por lo tanto, una mayor similitud del coseno.

Los embeddings de Azure OpenAI se basan en la similitud del coseno para calcular la similitud entre documentos y una consulta.

### **Aplicaciones**

Se pueden crear embeddings para todos los diferentes tipos de datos, incluyendo imágenes, audio, video y texto. En este notebook, veremos la generación de embeddings para archivos de texto y CSV.

Hay muchas aplicaciones en las que los embeddings pueden ser útiles. Por ejemplo, digamos que quieres clasificar un fragmento de texto. Una vez que se generan los embeddings, se pueden insertar en un modelo de aprendizaje automático para predecir la etiqueta correcta. Además, puedes utilizar embeddings para la similitud en datos de series temporales, datos de gráficos o para perfiles de usuarios o productos. Un caso de uso muy popular es el que implica la búsqueda semántica. Si deseas recuperar documentos que sean muy relevantes para tu consulta, se pueden generar embeddings tanto para la consulta como para los documentos con el fin de obtener una respuesta precisa. Veremos un ejemplo de esto en el Desafío 4.

## 2. Comencemos con la Implementación

Necesitarás importar los módulos necesarios. Las siguientes celdas son pasos de configuración de claves que ya completaste en los desafíos anteriores.

In [1]:
! pip install num2words
! pip install plotly
! pip install "openai==0.28.1" 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
  Attempting uninstall: openai
    Found existing installation: openai 0.27.2
    Uninstalling openai-0.27.2:
      Successfully uninstalled openai-0.27.2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import openai
import os
import re 
import requests
import sys
from num2words import num2words 
import pandas as pd 
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity 
import tiktoken
from dotenv import load_dotenv
from tenacity import retry, wait_random_exponential, stop_after_attempt
load_dotenv() 

True

Configura tu entorno para acceder a tus claves de Azure OpenAI. Consulta tu recurso de Azure OpenAI en el Portal de Azure para obtener información sobre tu punto de conexión y claves de Azure OpenAI.

Por razones de seguridad, almacena tu información sensible en un archivo .env.

In [3]:
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_version = os.getenv("OPENAI_API_VERSION")
embedding_model=os.getenv("EMBEDDING_MODEL_NAME")

## 3. Generar Embeddings en texto

#### Tarea #1 del Estudiante:
Utiliza la clase Azure OpenAI Embeddings para crear un embedding para el texto de entrada mostrado a continuación.

In [4]:

input="I would like to order a pizza"

def get_text_embedding(text, model=embedding_model):
    try:
        response = openai.Embedding.create(
            input=text,
            engine=model
        )
        return response['data'][0]['embedding']
    except Exception as e:
        print(f"Error al obtener el embedding: {e}")
        return None

embedding = get_text_embedding(input)

# Mostrar el resultado
if embedding:
    print("Embedding generado con éxito:")
    print(embedding)
    print("Tamaño: ")
    len(embedding)
else:
    print("No se pudo generar el embedding.")

Embedding generado con éxito:
[0.006318299565464258, -0.013745072297751904, -0.013720439746975899, -0.013313999399542809, -0.020124953240156174, 0.008233495056629181, -0.012045412324368954, -0.005219063255935907, 0.0072851344011723995, -0.018018852919340134, 0.03419025242328644, -0.005767141468822956, 0.004670984577387571, -0.008449031971395016, 0.00982846599072218, -0.006336773745715618, 0.02995341829955578, -0.00976688414812088, 0.011312589049339294, -0.0076546259224414825, 0.0001417345047229901, -0.013782021589577198, 0.01580190658569336, -0.00871383398771286, -0.016873430460691452, 0.002895887242630124, 0.004049007780849934, 0.010899989865720272, -0.0045293462462723255, -0.004520108923316002, 0.04226979240775108, 0.015247670002281666, -0.017329135909676552, -0.025248564779758453, -0.017772525548934937, 0.0025417914148420095, -0.002714220667257905, -0.01022258959710598, -0.002455576788634062, -0.028795680031180382, 0.01828981377184391, 0.0039042902644723654, 0.004889599978923798, -0

El método `openai.Embedding.create()` tomará una lista de texto - aquí tenemos una sola oración - y luego devolverá una lista que contiene un solo embedding. Puedes utilizar estos embeddings al buscar, proporcionar recomendaciones, clasificación y más.

### 3.1 Generar Embeddings para un archivo CSV

#### Tarea #2 del Estudiante:
Ingresa la ruta del archivo `Automobile.csv` que puedes encontrar en la carpeta `/data`. Ejecuta las celdas a continuación.

In [5]:
df=pd.read_csv(os.path.join(os.getcwd(),r'../data/Automobile.csv'))
df

,name,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,70,usa
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,70,usa
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,70,usa
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,70,usa
4,ford torino,17.0,8,302.0,140.0,3449,10.5,70,usa
...,...,...,...,...,...,...,...,...,...
393,ford mustang gl,27.0,4,140.0,86.0,2790,15.6,82,usa
394,vw pickup,44.0,4,97.0,52.0,2130,24.6,82,europe
395,dodge rampage,32.0,4,135.0,84.0,2295,11.6,82,usa
396,ford ranger,28.0,4,120.0,79.0,2625,18.6,82,usa


In [6]:
shortened_df = df[['name', 'mpg', 'origin']]
shortened_df

,name,mpg,origin
0,chevrolet chevelle malibu,18.0,usa
1,buick skylark 320,15.0,usa
2,plymouth satellite,18.0,usa
3,amc rebel sst,16.0,usa
4,ford torino,17.0,usa
...,...,...,...
393,ford mustang gl,27.0,usa
394,vw pickup,44.0,europe
395,dodge rampage,32.0,usa
396,ford ranger,28.0,usa


In [7]:
tokenizer = tiktoken.get_encoding("cl100k_base")
shortened_df['n_tokens'] = shortened_df["name"].apply(lambda x: len(tokenizer.encode(x)))
shortened_df = shortened_df[shortened_df.n_tokens<8192]
len(shortened_df)

/tmp/ipykernel_36672/3537330069.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortened_df['n_tokens'] = shortened_df["name"].apply(lambda x: len(tokenizer.encode(x)))


398

In [8]:
shortened_df

,name,mpg,origin,n_tokens
0,chevrolet chevelle malibu,18.0,usa,6
1,buick skylark 320,15.0,usa,7
2,plymouth satellite,18.0,usa,3
3,amc rebel sst,16.0,usa,5
4,ford torino,17.0,usa,2
...,...,...,...,...
393,ford mustang gl,27.0,usa,4
394,vw pickup,44.0,europe,2
395,dodge rampage,32.0,usa,3
396,ford ranger,28.0,usa,2


In [9]:
sample_encode = tokenizer.encode(shortened_df.name[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'che', b'vrolet', b' che', b'velle', b' mal', b'ibu']

In [10]:
len(decode)
shortened_df['ada-v2'] = shortened_df['name'].apply(lambda x : get_embedding(x, engine = embedding_model)) 

In [11]:
shortened_df

,name,mpg,origin,n_tokens,ada-v2
0,chevrolet chevelle malibu,18.0,usa,6,"[-0.0343162938952446, 0.0007342947646975517, 0..."
1,buick skylark 320,15.0,usa,7,"[-0.00984190870076418, -0.0031548449769616127,..."
2,plymouth satellite,18.0,usa,3,"[-0.019997337833046913, 0.016940420493483543, ..."
3,amc rebel sst,16.0,usa,5,"[-0.023981362581253052, -0.00803937017917633, ..."
4,ford torino,17.0,usa,2,"[-0.0385330505669117, -0.02776527777314186, -0..."
...,...,...,...,...,...
393,ford mustang gl,27.0,usa,4,"[-0.04225892201066017, -0.0038522197864949703,..."
394,vw pickup,44.0,europe,2,"[-0.02390824444591999, -0.015644479542970657, ..."
395,dodge rampage,32.0,usa,3,"[-0.024077117443084717, -0.005027948413044214,..."
396,ford ranger,28.0,usa,2,"[-0.014123061671853065, -0.02124483324587345, ..."


Los embeddings generados a partir del archivo CSV pueden usarse para realizar búsquedas. Puedes calcular la similitud del coseno entre un embedding de consulta y los embeddings del archivo CSV. Luego, puedes clasificar los resultados de la búsqueda según lo más relevante para la consulta. Veremos una aplicación de embeddings en el Desafío 4.

## Criterios de Éxito

Para completar este desafío con éxito:

* Demuestra comprensión de embeddings trabajando con diferentes entradas.